In [ ]:
%run ./utils.ipynb

In [ ]:
sz = 256
max_crop_sz = int(sz * 0.9)

In [ ]:
trn_tfms = albumentations.Compose([
    albumentations.HorizontalFlip(),
    albumentations.RandomSizedCrop((max_crop_sz, max_crop_sz), sz, sz, interpolation=1, p=0.5),
    albumentations.IAAAffine(rotate=10, p=0.5, mode='edge'),
    albumentations.Blur()
])

In [ ]:
db = get_data_bunch(sz=sz, bs=40, part=0, trn_tfms=trn_tfms)

In [ ]:
db.train_dl.dl.dataset.check_tfms(1)

In [ ]:
db.train_dl.dl.dataset.check_tfms(1)

In [ ]:
db.valid_dl.dl.dataset.check_tfms(15)

In [ ]:
# Hyperparameters
mult = 2 # channel multiplier for transposed convolutions
k = 4 # kernel size of transposed convolutions, needs to be a multiple of 2
folds_to_train = range(n_splits)

In [ ]:
def get_learner(db):
    res34 = pretrainedmodels.resnet34()
    base = nn.Sequential(*list(res34.children())[:-2])
    m = Unet34Like(base)
    m.cuda(default_device);
    learn = Learner(db, m, true_wd=True, loss_fn=bce_loss, layer_groups=split_model_idx(m, [61, 91]), opt_fn=lambda x: optim.SGD(x))
#     learn = Learner(db, m, true_wd=True, loss_fn=loss, layer_groups=split_model_idx(m, [61, 91]), opt_fn=AdamW)
    learn.metrics = [accuracy_thresh, dice, iou_pytorch]
    learn.callbacks = [SaveBest()]
    return learn

In [ ]:
class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, k, stride=2, padding=(k-2)//2)
        self.bn = nn.BatchNorm2d(n_out)
        self.out_channels = n_out
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p))
    
class Unet34Like(nn.Module):        
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.sfs = [SaveFeatures(rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(512,256,192 * mult)
        self.up2 = UnetBlock(192 * mult,128,96 * mult)
        self.up3 = UnetBlock(96 * mult,64,32 * mult)
        self.up4 = UnetBlock(32 * mult,64,32 * mult)
        self.up5 = UnetBlock(32 * mult,3,16 * mult)
        
        self.se1 = scSELayer(self.up1.out_channels)
        self.se2 = scSELayer(self.up2.out_channels)
        self.se3 = scSELayer(self.up3.out_channels)
        self.se4 = scSELayer(self.up4.out_channels)
        self.se5 = scSELayer(self.up5.out_channels)
        
        self.se_feat0 = scSELayer(64)
        self.se_feat1 = scSELayer(64)
        self.se_feat2 = scSELayer(128)
        self.se_feat3 = scSELayer(256)
        
        self.hc1 = HCBlock(self.up1.out_channels, out_sz=sz)
        self.hc2 = HCBlock(self.up2.out_channels, out_sz=sz)
        self.hc3 = HCBlock(self.up3.out_channels, out_sz=sz)
        self.hc4 = HCBlock(self.up4.out_channels, out_sz=sz)
        
        self.hc_comb = nn.Conv2d(64, 8, 3, padding=1)
        self.hc_bn = nn.BatchNorm2d(8)
        
        self.up6 = nn.ConvTranspose2d(16 * mult + 8, 1, 1)
        
    def forward(self,x):
        inp = x
        x = F.relu(self.rn(x))
        x = self.up1(x, self.se_feat3(self.sfs[3].features))
        x = self.se1(x)
        hc1 = self.hc1(x)
        
        x = self.up2(x, self.se_feat2(self.sfs[2].features))
        x = self.se2(x)
        hc2 = self.hc2(x)
        
        x = self.up3(x, self.se_feat1(self.sfs[1].features))
        x = self.se3(x)
        hc3 = self.hc3(x)
        
        x = self.up4(x, self.se_feat0(self.sfs[0].features))
        x = self.se4(x)
        hc4 = self.hc4(x)
        
        x = self.up5(x, inp)
        x = self.se5(x)
        
        hc = self.hc_comb(torch.cat((hc1, hc2, hc3, hc4), dim=1))
        hc = self.hc_bn(F.relu(hc))
        x = torch.cat((x, hc), dim=1)
        x = self.up6(x)
        return x[:,0]
    
    def close(self):
        for sf in self.sfs: sf.remove()

In [ ]:
%%time
# full size

for fold in folds_to_train:
    print(f'### Starting to train fold {fold} ###')
    db = get_data_bunch(sz=256, bs=40, part=fold, trn_tfms=trn_tfms)
    learn = get_learner(db)
    learn.freeze_to(2)
    learn.fit_one_cycle(3, 1e-1)
    learn.freeze_to(1)
    learn.fit_one_cycle(6, 1e-1)
    learn.unfreeze()
    learn.fit_one_cycle(40, 5e-2)
    learn.loss_fn = lovasz_loss
    learn.fit_one_cycle(40, 1e-2)
    learn.save(f'{name}_fold{fold}')

In [ ]:
%%time

upside_down = False

for fold in folds_to_train:
    db = get_data_bunch(sz=256, bs=40, part=fold, trn_tfms=trn_tfms)
    learn = get_learner(db)
    
    learn.load(f'{name}_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    print(f'Fold {fold} val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/val_targs_{name}_fold{fold}', val_targs)
    np.save(f'/home/radek/db/salt/test_preds_{name}_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    
    learn.load(f'{name}_best_iou_fold{fold}')
    val_preds, val_targs = predict_with_targs_and_TTA(learn.model, db.valid_dl, upside_down)
    test_preds = predict_with_TTA(learn.model, db.test_dl, upside_down)
    
    val_preds, test_preds = normalize_t(val_preds, val_targs, test_preds)
    print(f'Fold {fold} best iou val acc: {accuracy_np(val_preds, val_targs)}, iou: {iou_metric(val_targs, val_preds > 0.5)}')
    
    np.save(f'/home/radek/db/salt/val_preds_{name}_best_iou_fold{fold}', val_preds)
    np.save(f'/home/radek/db/salt/test_{name}_best_iou_fold{fold}', test_preds)
    del val_preds, val_targs, test_preds
    
    learn.model.close()
    del learn

In [ ]:
%%time
ys = []
preds = []
test_preds = np.zeros((18000, 256, 256))
for fold in folds_to_train:
    y = np.load(f'/home/radek/db/salt/val_targs_{name}_fold{fold}.npy')
    val_preds = np.load(f'/home/radek/db/salt/val_preds_{name}_fold{fold}.npy')
    preds.append(val_preds)
    ys.append(y)
    test_pred = np.load(f'/home/radek/db/salt/test_preds_{name}_fold{fold}.npy')
    test_preds += test_pred / len(folds_to_train)

np.save(f'/home/radek/db/salt/val_preds_{name}.npy', np.concatenate(preds))
np.save(f'/home/radek/db/salt/val_targs_{name}.npy', np.concatenate(ys))
np.save(f'/home/radek/db/salt/test_preds_{name}.npy', test_preds)

In [ ]:
%%time
for fold in folds_to_train:
    val_preds = np.load(f'/home/radek/db/salt/val_preds_{name}_fold{fold}.npy')
    val_targs = np.load(f'/home/radek/db/salt/val_targs_{name}_fold{fold}.npy')
    print(f'Part {fold}: {accuracy_np(val_preds, val_targs)}, {iou_metric(val_targs, val_preds > 0.5)}, {best_preds_t(val_preds, val_targs)}')

In [ ]:
val_preds = np.load(f'/home/radek/db/salt/val_preds_{name}.npy')
val_targs = np.load(f'/home/radek/db/salt/val_targs_{name}.npy')

In [ ]:
iou_metric(val_targs, val_preds > 0.5)

In [ ]:
%run diagnostics.ipynb

In [ ]:
test_preds = np.load(f'/home/radek/db/salt/test_preds_{name}.npy')

In [ ]:
preds_to_sub(test_preds, db.test_dl.dl.dataset.x, 0.486, 60, name)

In [ ]:
!kaggle competitions submit -c tgs-salt-identification-challenge -f ../subs/{name}.csv.gz  -m {name}